In [1]:
import re
from statistics import mean

import re
from statistics import mean

# Read logs from files
with open("yolo.txt", "r") as f:
    yolo_log = f.read()

with open("godot.txt", "r") as f:
    godot_log = f.read()


# --- Parse YOLO log ---

yolo_pattern = re.compile(
    r"Speed: ([\d.]+)ms preprocess, ([\d.]+)ms inference, ([\d.]+)ms postprocess.*?\[Frame (\d+)\] Timestamp: ([\d.]+)",
    re.DOTALL
)

yolo_data = []
for match in yolo_pattern.finditer(yolo_log):
    preprocess, inference, postprocess, frame_num, timestamp = match.groups()
    total_time = float(preprocess) + float(inference) + float(postprocess)
    yolo_data.append((int(frame_num), float(timestamp), total_time))

# --- Parse Godot log ---
godot_pattern = re.compile(r"Parsed pose #(\d+) at (\d+) ms")
godot_data = [(int(idx), int(ts)) for idx, ts in godot_pattern.findall(godot_log)]

# --- Match frames and calculate delays ---
delays = []
matched_times = []

# Just for simplicity, assume matching is index-based
# In real use, you'd sync frame number or timestamp better
for i, (idx, godot_ts_ms) in enumerate(godot_data):
    if i >= len(yolo_data):
        break
    frame_num, yolo_ts_sec, yolo_total_time = yolo_data[i]
    godot_ts_sec = godot_ts_ms / 1000.0
    delay = godot_ts_sec - yolo_ts_sec
    delays.append(delay)
    matched_times.append(yolo_total_time)

# --- Results ---
print(f"Average YOLO processing time: {mean(matched_times):.2f} ms")
print(f"Average delay between YOLO and Godot: {mean(delays)*1000:.2f} ms")


Average YOLO processing time: 13.17 ms
Average delay between YOLO and Godot: 50.09 ms
